# Compiling RCS to Clifford+RZ (With Qrack)

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 8
max_magic = 8
shots = 1024

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import collections
import math
import random
from pyqrack import QrackSimulator, Pauli
from mitiq import zne

# Change these according to your OpenCL devices and system specifications:
# os.environ['QRACK_MAX_PAGE_QB']='27'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'
# os.environ['QRACK_MAX_ALLOC_MB']='147456,15872'
# os.environ['QRACK_MAX_PAGING_QB']='30'
# os.environ['QRACK_MAX_CPU_QB']='30'

In [3]:
sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(q):
    return lambda circ: circ.s(q)

def x_to_z(q):
    return lambda circ: circ.h(q)

def _y_to_z(circ, q):
    circ.s(q)
    circ.h(q)

def y_to_z(q):
    return lambda circ: _y_to_z(circ, q)    

def y_to_x(q):
    return lambda circ: circ.adjs(q)

def z_to_x(q):
    return lambda circ: circ.h(q)

def _z_to_y(circ, q):
    circ.h(q)
    circ.s(q)

def z_to_y(q):
    return lambda circ: _z_to_y(circ, q)

def cx(q1, q2):
    return lambda circ: circ.mcx([q1], q2)

def cy(q1, q2):
    return lambda circ: circ.mcy([q1], q2)

def cz(q1, q2):
    return lambda circ: circ.mcz([q1], q2)

def acx(q1, q2):
    return lambda circ: circ.macx([q1], q2)

def acy(q1, q2):
    return lambda circ: circ.macy([q1], q2)

def acz(q1, q2):
    return lambda circ: circ.macz([q1], q2)

def swap(q1, q2):
    return lambda circ: circ.swap(q1, q2)

def _nswap(circ, q1, q2):
    circ.macz([q1], q2)
    circ.swap(q1, q2)
    circ.macz([q1], q2)

def nswap(q1, q2):
    return lambda circ: _nswap(circ, q1, q2)

def _pswap(circ, q1, q2):
    circ.macz([q1], q2)
    circ.swap(q1, q2)

def pswap(q1, q2):
    return lambda circ: _pswap(circ, q1, q2)    

def _mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.macz([q1], q2)

def mswap(q1, q2):
    return lambda circ: _mswap(circ, q1, q2)    

def iswap(q1, q2):
    return lambda circ: circ.iswap(q1, q2)

def iiswap(q1, q2):
    return lambda circ: circ.adjiswap(q1, q2)

def random_circuit(width, max_magic):
    max_magic = max_magic / 2
    t_count = 0
    
    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) }
    single_bit_mirrors = { 0: (x_to_z, y_to_z), 1: (y_to_x, z_to_x), 2: (z_to_y, x_to_y) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz
    two_bit_mirrors = swap, mswap, pswap, nswap, iiswap, iswap, cx, cy, cz, acx, acy, acz
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width

    circuit = []
    mirror = []
    
    for i in range(3 * width):
        # Single bit gates
        single_qubit_layer = []
        single_qubit_mirror = []
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)
            
            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            single_qubit_layer.append(gate(j))
            gate = single_bit_mirrors[bases[j]][directions[j]]
            single_qubit_mirror.append(gate(j))

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3

            # Rotate around local Z axis
            rnd = random.randint(0, 3)
            if rnd == 0:
                single_qubit_layer.append(lambda circ: circ.s(j))
                single_qubit_mirror.append(lambda circ: circ.adjs(j))
            elif rnd == 1:
                single_qubit_layer.append(lambda circ: circ.z(j))
                single_qubit_mirror.append(lambda circ: circ.z(j))
            elif rnd == 2:
                single_qubit_layer.append(lambda circ: circ.adjs(j))
                single_qubit_mirror.append(lambda circ: circ.s(j))
            
            if (t_count < max_magic) and (3 * width * width * random.random() / max_magic) < 1:
                angle = random.uniform(0, 2 * math.pi)
                single_qubit_layer.append(lambda circ: circ.r(Pauli.PauliZ, angle, j))
                single_qubit_mirror.append(lambda circ: circ.r(Pauli.PauliZ, -angle, j))
                t_count += 1

        circuit.append(single_qubit_layer)
        mirror.append(single_qubit_mirror)

        two_qubit_layer = []
        two_qubit_mirror = []

        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                rnd = random.randint(0, len(two_bit_gates) - 1)
                g = two_bit_gates[rnd]
                two_qubit_layer.append(g(b1, b2))
                g = two_bit_mirrors[rnd]
                two_qubit_mirror.append(g(b1, b2))

        circuit.append(two_qubit_layer)

    mirror.reverse()
    for i in range(len(mirror)):
        mirror[i].reverse()
    circuit = circuit + mirror
    
    return circuit

In [4]:
circ = random_circuit(width, max_magic)

In [5]:
qubits = list(range(width))

def execute(circuit, qsim):
    """Returns the expectation value for unsigned integer overall bit string."""

    for layer in circ:
        for gate in layer:
            gate(qsim)

    return sum(qsim.measure_shots(qubits, shots)) / shots

qsim = QrackSimulator(width)
qsim.set_weak_sampling(True)

noisy_value = execute(circ, qsim)
print("Potentially noisy value: ", noisy_value)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Potentially noisy value:  56.0


In [6]:
if noisy_value > 1e-6:
    print("Error with ZNE: ", zne.execute_with_zne(circ, execute))
else:
    print("No mitigation required. (Ideal value.)")

UnsupportedCircuitError: Could not determine the package of the input circuit.